Multi-Class Prediction of Obesity Risk

In [ ]:
#Importing the Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder

In [ ]:
#reding the file
df = pd.read_csv("train.csv")
df

In [ ]:
df.dtypes

In [ ]:
df.isna().sum()

In [ ]:
df.select_dtypes(include=['object']).columns

In [ ]:
for col in df.select_dtypes(include=['object']).columns: 
    print(f"{col} : {df[col].unique()}")

In [ ]:
# Assuming df is your DataFrame with 18 columns

# Calculate the number of rows and columns needed for subplots
num_rows = 6  # You can adjust this based on your preference
num_cols = 3

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 20))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Plot histograms for each column
for i, col in enumerate(df.columns):
    df[col].hist(ax=axes[i], bins=20)
    axes[i].set_title(col)

# Hide any remaining empty subplots
for i in range(len(df.columns), len(axes)):
    axes[i].axis('off')

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Calculate correlation matrix
df_dummies = pd.get_dummies(df)
df_dummies.corr()

In [ ]:
# Plot correlation heatmap
plt.figure(figsize=(20, 20))
sns.heatmap(df_dummies.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap (including Categorical Columns)')
plt.yticks(rotation=0)
plt.show()

In [ ]:
correlation_matrix = df_dummies.corr()

In [ ]:
# Assuming correlation_matrix is your correlation matrix DataFrame

# Create empty lists to store pairs
positive_correlation_pairs = []
negative_correlation_pairs = []

# Iterate through the correlation matrix
for i in range(len(correlation_matrix.columns)):
    for j in range(i + 1, len(correlation_matrix.columns)):
        correlation = correlation_matrix.iloc[i, j]
        if correlation >= 0.3:
            positive_correlation_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], correlation))
        elif correlation <= -0.3:
            negative_correlation_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], correlation))

# Convert lists to DataFrames if needed
positive_correlation_df = pd.DataFrame(positive_correlation_pairs, columns=['Feature 1', 'Feature 2', 'Correlation'])
negative_correlation_df = pd.DataFrame(negative_correlation_pairs, columns=['Feature 1', 'Feature 2', 'Correlation'])

In [ ]:
positive_correlation_df

In [ ]:
negative_correlation_df

In [ ]:
df_dummies

In [ ]:
# Ordinal Encoding
ordinal_encoder = OrdinalEncoder()
df['NObeyesdad'] = ordinal_encoder.fit_transform(df[['NObeyesdad']])
df

In [ ]:
ordinal_encoder = OrdinalEncoder()
df['SMOKE'] = ordinal_encoder.fit_transform(df[['SMOKE']])
df

In [ ]:
ordinal_encoder = OrdinalEncoder()
df['family_history_with_overweight'] = ordinal_encoder.fit_transform(df[['family_history_with_overweight']])
df

In [ ]:
for col in df.select_dtypes(include=['object']).columns: 
    ordinal_encoder = OrdinalEncoder()
    df[col] = ordinal_encoder.fit_transform(df[[col]])
df

Let's findout whethere there is noise in data or not?
Using clustering

In [ ]:
df = df.drop(columns=['id'])
df

In [ ]:
lable = df['NObeyesdad']
df = df.drop(columns=['NObeyesdad'])

In [ ]:
df

In [ ]:
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import numpy as np

# Assuming X is your feature matrix
X = df.select_dtypes(include=['number'])  # Select only numerical columns

# Specify DBSCAN parameters
epsilon = 5  # Maximum distance between points to be considered neighbors
min_samples = 5  # Minimum number of points required to form a cluster

# Initialize DBSCAN
dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)

# Fit DBSCAN to the data
dbscan.fit(X)

# Get cluster labels (-1 represents noise/outliers)
cluster_labels = dbscan.labels_
noise_indices = np.where(cluster_labels == -1)[0]
# Plot the clustering results
plt.figure(figsize=(8, 6))

# Plot points colored by cluster label
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=cluster_labels, cmap='viridis', s=50)

# Highlight noise points
plt.scatter(X.iloc[noise_indices, 0], X.iloc[noise_indices, 1], c='red', marker='x', label='Noise points')

plt.title('DBSCAN Clustering')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Assuming X is your feature matrix
X = df.select_dtypes(include=['number'])  # Select only numerical columns

# Specify DBSCAN parameters
epsilon = 5  # Maximum distance between points to be considered neighbors
min_samples = 5  # Minimum number of points required to form a cluster

# Initialize DBSCAN
dbscan = DBSCAN(eps=epsilon, min_samples=min_samples)

# Fit DBSCAN to the data
dbscan.fit(X)

# Get cluster labels (-1 represents noise/outliers)
cluster_labels = dbscan.labels_
noise_indices = np.where(cluster_labels == -1)[0]

# Assign cluster labels to the DataFrame
df['cluster_labels'] = cluster_labels

# Plot the clustering results using scatter plot matrix
sns.pairplot(df, diag_kind='kde', hue='cluster_labels', palette='viridis')
plt.title('DBSCAN Clustering')
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Assuming X is your feature matrix
# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to your data and transform it
X_normalized = scaler.fit_transform(X)

# Convert the normalized data back to a DataFrame (if needed)
df_normalized = pd.DataFrame(X_normalized, columns=X.columns)

# Display the normalized DataFrame
df_normalized

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

# Assuming X is your feature matrix and y is your target variable
# Initialize SelectKBest with the desired scoring function
selector = SelectKBest(score_func=f_classif, k=13)  # Adjust the number of features (k) as needed

# Fit the selector to your data and transform it
X_selected = selector.fit_transform(df_normalized, lable)

# Get the selected features
selected_features = X.columns[selector.get_support()]
selected_features

In [ ]:
X_FS = df_normalized[selected_features]
X_FS

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

# Assuming X is your feature matrix and y is your target variable
# Initialize StratifiedShuffleSplit
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Perform stratified sampling
for train_index, test_index in stratified_split.split(X_FS, lable):
    X_train, X_test = X_FS.iloc[train_index], X_FS.iloc[test_index]
    y_train, y_test = lable.iloc[train_index], lable.iloc[test_index]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest classifier
rf_classifier = RandomForestClassifier()

# Train the classifier on your data
rf_classifier.fit(X_train, y_train)

# Predict on test data
y_pred = rf_classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Assuming y_true are the true labels and y_pred are the predicted labels
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# Plot confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cmap='Blues', fmt='g', )
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Holy shit!

In [ ]:
#SVM
from sklearn.svm import SVC
svm_classifier = SVC(kernel='poly', C=1.0, gamma='scale')
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

# Not So good


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize Gradient Boosting classifier
gb_classifier = GradientBoostingClassifier()

# Train the classifier on your data
gb_classifier.fit(X_train, y_train)

# Predict on test data
y_pred = gb_classifier.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cmap='Blues', fmt='g', )
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Slightly better 

In [ ]:
from sklearn.linear_model import LogisticRegression

# Initialize Logistic Regression classifier
lr_classifier = LogisticRegression()

# Train the classifier on your data
lr_classifier.fit(X_train, y_train)

# Predict on test data
y_pred = lr_classifier.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

# The Worst

In [ ]:
testData = pd.read_csv("test.csv")
testData

In [ ]:
for col in testData.select_dtypes(include=['object']).columns: 
    ordinal_encoder = OrdinalEncoder()
    testData[col] = ordinal_encoder.fit_transform(testData[[col]])

In [ ]:
testData= testData[X_train.columns]
testData

In [ ]:
y_pred = gb_classifier.predict(testData)

In [ ]:
y_pred

In [ ]:
id = pd.read_csv("test.csv")
id

In [ ]:
id = id["id"]
id


In [ ]:
df_arr = pd.DataFrame(y_pred, columns=['NObeyesdad'])
df_arr

In [ ]:
# Assuming encoded_labels is your Series with encoded labels
# Define a dictionary mapping encoded labels to original labels
label_mapping = {0: 'Insufficient_Weight'
                 , 1: 'Normal_Weight'
                 , 2: 'Obesity_Type_I' 
                 , 3: 'Obesity_Type_II'
                 , 4: 'Obesity_Type_III'
                 , 5: 'Overweight_Level_I'
                 , 6: 'Overweight_Level_II'}


# Transform encoded labels back to original labels using the mapping
original_labels = df_arr["NObeyesdad"].map(label_mapping)

In [ ]:
original_labels

In [ ]:
original_labels = pd.DataFrame(original_labels, columns=['NObeyesdad'])
original_labels


In [ ]:
submission = pd.concat([id, original_labels], axis=1)
submission

In [ ]:
submission.to_csv('output.csv', index=False)